In [1]:
import random
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from operator import itemgetter

# Genetic Search Algorithm

In [11]:
def point_mutation(lineup, rate, person_lookup, formation, week):
    '''
    Description:
        Randomly change a bit inside of a particular child

    Input:
        lineup(list): list of players
        rate(dbl): the rate at which the mutation happens

    Output:
        child(list): the bitstring with possible mutations from the original
    '''
    defense, midfield, forward = formation
    gkp_match, def_match, mid_match, fwd_match = (True, True, True, True)

    
    child = lineup.copy()

    for idx, j in enumerate(lineup):
        if random.uniform(0, 1) < rate:
            if idx == 0:
                while gkp_match:
                    gkp_idx = int(random.sample(range((len(person_lookup[week]['GKP'])-1)), 1)[0])

                    if person_lookup[week]['GKP'][gkp_idx] == child[idx]:
                        continue
                    else:
                        gkp_match = False
                        child[idx] = person_lookup[week]['GKP'][gkp_idx]

            elif idx <= defense:
                while def_match:
                    def_idx = int(random.sample(range((len(person_lookup[week]['DEF'])-1)), 1)[0])

                    if person_lookup[week]['DEF'][def_idx] == child[idx]:
                        continue
                    else:
                        def_match = False
                        child[idx] = person_lookup[week]['DEF'][def_idx]
                
            elif idx <= (defense+midfield):
                while mid_match:
                    mid_idx = int(random.sample(range((len(person_lookup[week]['MID'])-1)), 1)[0])

                    if person_lookup[week]['MID'][mid_idx] == child[idx]:
                        continue
                    else:
                        mid_match = False
                        child[idx] = person_lookup[week]['MID'][mid_idx]

            else:
                while fwd_match:
                    fwd_idx = int(random.sample(range((len(person_lookup[week]['FWD'])-1)), 1)[0])

                    if person_lookup[week]['FWD'][fwd_idx] == child[idx]:
                        continue
                    else:
                        fwd_match = False
                        child[idx] = person_lookup[week]['FWD'][fwd_idx]


    return child

def crossover(parent1, parent2, rate, formation):
    '''
    Description:
        mutate a new child based off of the combination of two parents.

    Input:
        parent1(list): list of bitstrings
        parent2(list): list of bitstrings
        rate(dbl): the rate at which the muatation happens. If a random number
        is greater than the rate, no mutation will occur.

    Output:
        a possible combination of the two parents.
    '''
    defense, midfield, forward = formation

    def_same_flag, def_same_count = (True, 0)
    mid_same_flag, mid_same_count = (True, 0)
    fwd_same_flag, fwd_same_count = (True, 0)

    # If the mutation rate is lower than the overall rate,
    # take part of parent 1 and part of parent two and combine them 
    # into a new child

    # Goalie
    if random.uniform(0, 1) >= rate:
        return parent1['lineup']
    else:
        if random.randint(0,1) == 0:
            new_gkp = [parent1['lineup'][0]]
        else:
            new_gkp = [parent2['lineup'][0]]

        # Defense
        parent1_def = random.randint(0, defense)
        parent2_def = defense - parent1_def 

        if parent1_def == 0:
            new_def = parent2['lineup'][1:(defense+1)]
        elif parent2_def == 0:
            new_def = parent1['lineup'][1:(defense+1)]
        else:
            while (def_same_flag == True) and (def_same_count < 4):
                def_random_parent1 = random.sample(range(0, (defense-1)), parent1_def)
                def_random_parent2 = random.sample(range(0, (defense-1)), parent2_def)

                def_sample_parent_1 = list(np.array(parent1['lineup'][1:(defense+1)])[def_random_parent1])
                def_sample_parent_2 = list(np.array(parent2['lineup'][1:(defense+1)])[def_random_parent2])

                def_same_check = sum([i in def_sample_parent_2 for i in def_sample_parent_1])

                if def_same_check > 0:
                    def_same_count += 1
                else:
                    def_same_flag = False
                    new_def = def_sample_parent_1 + def_sample_parent_2
        
        # Midfield
        parent1_mid = random.randint(0, midfield)
        parent2_mid = midfield - parent1_mid 

        if parent1_mid == 0:
            new_mid = parent2['lineup'][(defense+1):(defense+midfield+1)]
        elif parent2_mid == 0:
            new_mid = parent1['lineup'][(defense+1):(defense+midfield+1)]
        else:
            while (mid_same_flag == True) and (mid_same_count < 4):
                mid_random_parent1 = random.sample(range(0, (midfield-1)), parent1_mid)
                mid_random_parent2 = random.sample(range(0, (midfield-1)), parent2_mid)

                mid_sample_parent_1 = list(np.array(parent1['lineup'][(defense+1):(defense+midfield+1)])[mid_random_parent1])
                mid_sample_parent_2 = list(np.array(parent2['lineup'][(defense+1):(defense+midfield+1)])[mid_random_parent2])

                mid_same_check = sum([i in mid_sample_parent_2 for i in mid_sample_parent_1])

                if mid_same_check > 0:
                    mid_same_count += 1
                else:
                    mid_same_flag = False
                    new_mid = mid_sample_parent_1 + mid_sample_parent_2

        # Forward
        if forward == 1:
            if random.randint(0, 1) == 0:
                new_fwd = list(parent1['lineup'][-1:])
            else:
                new_fwd = list(parent2['lineup'][-1:])
        else:
            parent1_fwd = random.randint(0, forward)
            parent2_fwd = forward - parent1_fwd 

            if parent1_fwd == 0:
                new_fwd = parent2['lineup'][(defense+midfield+1):]
            elif parent2_fwd == 0:
                new_fwd = parent1['lineup'][(defense+midfield+1):]
            else:
                while (fwd_same_flag == True) and (fwd_same_count < 5):
                    fwd_random_parent1 = random.sample(range(0, (forward-1)), parent1_fwd)
                    fwd_random_parent2 = random.sample(range(0, (forward-1)), parent2_fwd)

                    fwd_sample_parent_1 = list(np.array(parent1['lineup'][(defense+midfield+1):])[fwd_random_parent1])
                    fwd_sample_parent_2 = list(np.array(parent2['lineup'][(defense+midfield+1):])[fwd_random_parent2])

                    fwd_same_check = sum([i in fwd_sample_parent_2 for i in fwd_sample_parent_1])

                    if fwd_same_check > 0:
                        fwd_same_count += 1
                    else:
                        fwd_same_flag = False
                        new_fwd = fwd_sample_parent_1 + fwd_sample_parent_2
        
        if fwd_same_count >= 4:
            if random.randint(0, 1) == 0:
                new_fwd = parent1['lineup'][(defense+midfield+1):]
            else:
                new_fwd = parent2['lineup'][(defense+midfield+1):]

        if mid_same_count >= 4 :
            if random.randint(0, 1) == 0:
                new_mid = parent1['lineup'][(defense+1):(defense+midfield+1)]
            else:
                new_mid = parent2['lineup'][(defense+1):(defense+midfield+1)]

        if def_same_count >= 4:
            if random.randint(0, 1) == 0:
                new_def = parent1['lineup'][1:(defense+1)]
            else:
                new_def = parent2['lineup'][1:(defense+1)]

        return new_gkp + new_def + new_mid + new_fwd

def reproduce(selected, pop_size, p_cross, p_mutation, formation, person_lookup, week):
    '''
    Description:
        Take parent strings a combine them combine/mutate them into children strings.

    Input:
        selected(list): list of dictionaries that are potential best strings
        pop_size(int): number of candiadtes to consider
        p_cross(dbl): rate at which the parent to parent crossover happens
        p_mutation(dbl): rate at which the single point mutation will happen

    Output:
        children(list): list of all the children created from the parents.
    '''
    children = []
    # Selects the items before (Odd indicies) and after (even indicies) 
    # for mutation later
    for i, p1 in enumerate(selected):
        if i % 2 == 0:
            p2 = selected[i+1]
        else:
            p2 = selected[i-1]

        if i == (len(selected)-1):
            p2 = selected[0]

        child = {}
        # Tries to combine the two parents into new children
        child['lineup'] = crossover(p1, p2, p_cross, formation)

        # Randomly mutates points within the child
        # child['lineup'] = point_mutation(child['lineup'], p_mutation, person_lookup, formation, week)
        children.append(child)
    
        if len(children) == pop_size:
            break

    return children

def binary_tournament(pop):
    '''
    Description:
        The function will randomly select two different lists from the population pool.
        It will then compared the cost of the two list against each other. Which ever
        list has a better cost, they get selected and move on in the selection process.

    Input:
        pop(list): List of dictionaries containing the cost and bits for each item under
        consideration

    Output:
        dictionary contating the list that won the "tournament"

        {bitstring: [],
         fitness: int}
    '''
    # Randomly select two bitstrings from the overall list
    i, j = random.randint(0, (len(pop)-1)), random.randint(0, (len(pop)-1))

    # Check to see if the indicies are the same
    while i == j:
        j =  random.randint(0, len(pop)-1)

    # Select the string with the highest cost to move forward.
    if pop[i]['fitness'] >= pop[j]['fitness']:
        return pop[i]
    else: 
        return pop[j]

def onemax(lineup, player_scores_weeks, week):
    '''
    The cost of the algorithm is the sum of the player costs 
    '''
    scores = [player_scores_weeks[week][i] for i in lineup]
    return sum(scores)

def random_lineup(formation, person_lookup, week):
    '''
    Description:
        Create a random list of players

    Input:
        formation(tuple): tuple containing the number of defenders, midfielders and forwards in the formation
        person_lookup(dict): Dictionary containing the names of the players for each position

    Output:
        A list of randomly generated players.
    '''
    goalie = 1
    defense, midfield, forward = formation

    goalie_idx = random.sample(range(0, (len(person_lookup[week]['GKP'])-1)), 1)
    defense_idx = random.sample(range(0, (len(person_lookup[week]['DEF'])-1)), defense)
    midfield_idx = random.sample(range(0, (len(person_lookup[week]['MID'])-1)), midfield)
    forward_idx = random.sample(range(0, (len(person_lookup[week]['FWD'])-1)), forward)

    goalies = list(person_lookup[week]['GKP'][goalie_idx])
    defenders = list(person_lookup[week]['DEF'][defense_idx])
    midies = list(person_lookup[week]['MID'][midfield_idx])
    forwards = list(person_lookup[week]['FWD'][forward_idx])

    return goalies + defenders + midies + forwards

def search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week):
    '''
Description:
    This function contians the main logic of the program. It will search through
    the various combinations of children and parents to find a soultion that will
    optimize the given cost functions. In this case, the optimal solution is a 
    list of all ones.

Input:
    max_gens(int): The number of generations to loop through to find an optimal
    solution.
    num_bits(int): The number of bits contained in the list
    pop_size(int): The number of children to be children to be considered against the parent.
    p_crossover(dbl): THe probability of keeping the parent vs mixing aprent and child.
        ex) p_crossover = .99; probability of keeping parent to next generation = .01
    p_mutation(dbl): The probability of changing a single position in a list.

Output:
    best(list): A list of a possible optimal solution.  
    '''
    early_stop = 0

    # Create a random set of possible solutions
    population = [{'lineup':random_lineup(formation, person_lookup, week)} for i in range(pop_size)]
    
    # Find the "cost" of each colution
    for idx,_ in enumerate(population):
        population[idx]['fitness'] = onemax(population[idx]['lineup'], player_scores_weeks, week)

    # Sort the lists to find the solution with the highest cost, and keep the largest as the best solution
    best = sorted(population, key=itemgetter('fitness'), reverse=True)[0]

    # # Loop through the number of generations
    for gen in range(max_gens):
        old_fitness = best['fitness']

        # Randomly select possible offspring
        selected = [binary_tournament(population) for _ in range(len(population))]
        # Reproduce children based off of the parent 
        children = reproduce(selected, pop_size, p_crossover, p_mutation, formation, person_lookup, week)

        for idx, _ in enumerate(children):
            children[idx]['fitness'] = onemax(children[idx]['lineup'], player_scores_weeks, week)

        sort_children =  sorted(children, key=itemgetter('fitness'), reverse=True) 

        # If the best child has a higher cost than current, replace the current with child
        if sort_children[0]['fitness'] >= best['fitness']:
            best = sort_children[0].copy()

        population = sort_children.copy()

        print('Gen: {0}, Fitness: {1}, Best: {2} '.format(gen, best['fitness'], best['lineup']))

        if old_fitness == best['fitness']:
            early_stop += 1
        else:
            early_stop = 0

        if early_stop == 5:
            break

    return best

# Data Prep

In [3]:
def normalize_data(col):
    '''
    Helper function to normalize a array of data betwene 0 (minimum) and 1 (maximum)

    Input:
        col(list/series): columns from a dataframe that is to be normalized

    Output:
        Normalized array with values between 0 and 1
    '''
    return (col-np.min(col))/(np.max(col)-np.min(col))

def scoring(df, label, week):
    '''
    This function will go through and calculate the scores for each player at their 
    given poisition. The scores are used by the algorithms cost function to figure out
    what the total "cost" would be to add a patricular player to the roster.

    Input:
        df(pd.dataframe): List of all the players for a given position
        label(str): label for the position of the players in ther
        week(int): the week of the season the daya is in.
            Week 0 is "preseaon" and uses projected numbers. The coefficients
            of the equations have to be addujsted accordingly
    
    Output:
        a dataframe with the calculated scores appended on the end.
    '''

    data = df.copy()

    # a Multiplier for the coefficients (There are 38 games in the average EPL season)
    if week == 0:
        ceof_multi = 38
    else:
        ceof_multi = 1

    # Equation to calculate the goalkeepers scores
    if label == 'GKP':
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Scores'] = 10*ceof_multi - \
                          1.5*ceof_multi*data['Goals_conceded'] + \
                          2.0*ceof_multi*data['Penalties_saved'] + \
                          ceof_multi*data['Saves'] - \
                          3.0*ceof_multi*data['Red_cards'] + \
                          2.5*ceof_multi*data['Norm_Influence']

        # If the score is equal to the intercept, then the player has no stats to be considered
        # Therefore they will not be on a roster.
        
        return data[data['Scores'] != 10*ceof_multi]

    # Equation to calculate the defenders scores
    elif label == 'DEF':
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Norm_Threat'] = 5.0*normalize_data(data['Threat'])
        data['Norm_Creativity'] = 5.0*normalize_data(data['Creativity'])
        data['Scores'] = 10*ceof_multi - \
                            ceof_multi*data['Goals_conceded'] + \
                          5*ceof_multi*data['Goals_scored'] + \
                          2.5*ceof_multi*data['Norm_Influence'] - \
                          1.5*ceof_multi*data['Norm_Threat'] + \
                          1.5*ceof_multi*data['Norm_Creativity'] - \
                          2.0*ceof_multi*data['Red_cards'] + \
                          ceof_multi*data['Assists'] - \
                          2.0*ceof_multi*data['Own_goals']  
        return data[data['Scores'] != 10*ceof_multi]     

    # Equation to calculate the Midfielders scores
    elif label == 'MID':
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Norm_Creativity'] = 5.0*normalize_data(data['Creativity'])
        data['Norm_Threat'] = 5.0*normalize_data(data['Threat'])
        data['Scores'] = 10*ceof_multi - \
                        ceof_multi*data['Goals_conceded'] + \
                        2.0*ceof_multi*data['Goals_scored'] + \
                        2.5*ceof_multi*data['Norm_Influence'] + \
                        1.5*ceof_multi*data['Norm_Threat'] + \
                        5.0*ceof_multi*data['Norm_Creativity'] - \
                        2.0*ceof_multi*data['Red_cards'] + \
                        2.0*ceof_multi*data['Assists'] - \
                        2.0*ceof_multi*data['Own_goals']

        return data[data['Scores'] != 10*ceof_multi] 

    # Equation to calculate the Forwards scores
    else:
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Norm_Creativity'] = 5.0*normalize_data(data['Creativity'])
        data['Norm_Threat'] = 5.0*normalize_data(data['Threat'])
        data['Scores'] = 10*ceof_multi + \
                            2.0*ceof_multi*data['Goals_scored'] + \
                            2.5*ceof_multi*data['Norm_Influence'] + \
                            1.5*ceof_multi*data['Norm_Creativity'] + \
                            5.0*ceof_multi*data['Norm_Threat'] - \
                            2.5*ceof_multi*data['Red_cards'] + \
                            ceof_multi*data['Assists']
        return data[data['Scores'] != 10*ceof_multi] 

In [4]:
PATH = 'https://raw.githubusercontent.com/DanielBrooks253/Kaggle/main/Soccer_Team/Data/'
filenames = ['FPL_2018_19_Wk0.csv', # Projected Scores
             'FPL_2018_19_Wk1.csv',
             'FPL_2018_19_Wk2.csv',
             'FPL_2018_19_Wk3.csv',
             'FPL_2018_19_Wk4.csv',
             'FPL_2018_19_Wk5.csv',
             'FPL_2018_19_Wk6.csv',
             'FPL_2018_19_Wk7.csv',
             'FPL_2018_19_Wk8.csv',
             'FPL_2018_19_Wk9.csv',
             'FPL_2018_19_Wk10.csv',
             'FPL_2018_19_Wk11.csv']

person_lookup = {'DEF': [],
                 'GKP': [],
                 'MID': [],
                 'FWD': []}

# Get the individual weeks and also the overall combined dataset
weeks = [pd.read_csv(PATH + i) for i in filenames]
adjust_names = weeks.copy()

# Some of the names of the players repeat.
# This causes issues when trying to create dictionary keys later on.
# THerefore I concatenated the team name with the last name to make a unique key.
for idx, _ in enumerate(adjust_names):
    adjust_names[idx]['New_Name'] = adjust_names[idx]['Name'] + ', ' + adjust_names[idx]['Team']  

In [5]:
# Get the scores for all of the positions for all of the available weeks
list_player_score_df = [pd.concat(list(map(lambda x: scoring(adjust_names[i][adjust_names[i]['Position'] == x], x, i), ['DEF', 'FWD', 'GKP', 'MID'])), axis=0)
                           for i in range(0, 12)]

# Create a list of dictionaries for each week.
# The dictionary maps the players name to their scores
player_scores_weeks = [dict(zip(i['New_Name'], i['Scores'])) for i in list_player_score_df]

# Filter the weeks to only players who played 
player_score_keys = [list(i.keys()) for i in player_scores_weeks]
filtered_weeks = [i[i['New_Name'].isin(j)] for i,j in zip(adjust_names, player_score_keys)]

In [6]:
# Get the mapping from the position to the person
filter_person = [list(map(lambda x: filtered_weeks[i][filtered_weeks[i]['Position'] == x], ['DEF', 'FWD', 'GKP', 'MID'])) for i in range(0, 12)]
person_lookup = [{j['Position'].unique()[0]:np.array(j['New_Name']) for j in i} for i in filter_person]

# Different Formations (# Defenders - # Midfielders - # Forwards)

## 4-3-3

In [32]:
formation = (4,3,3)
max_gens = 100
pop_size = 30
p_crossover = .99
p_mutation = 1.0/64
week = 0

foo = search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week)

print('\n ***************************************************************************** \n' 
'                        Best Lineup for week {0}                                    \n' 
'                         Formation: {1}-{2}-{3}                                     \n' 
'***************************************************************************** \n'                    
'GKP: {4} \n'
'DEF: {5} \n'
'MID: {6} \n'
'FWD: {7}'.format(week, formation[0], formation[1], formation[2], foo['lineup'][0], 
                                                                  foo['lineup'][1:(formation[0]+1)],
                                                                  foo['lineup'][(formation[0]+1):(formation[0]+formation[1]+1)],
                                                                  foo['lineup'][(formation[0]+formation[1]+1):]))

Gen: 0, Fitness: 14473.05164425615, Best: ['Ryan, BHA', 'Evans, LEI', 'Schelotto, BHA', 'Long, BUR', 'Aurier, TOT', 'Ritchie, NEW', 'Sane, MCI', 'Drinkwater, CHE', 'Lukaku, MUN', 'Arnautovic, WHU', 'Kane, TOT'] 
Gen: 1, Fitness: 14473.05164425615, Best: ['Ryan, BHA', 'Evans, LEI', 'Schelotto, BHA', 'Long, BUR', 'Aurier, TOT', 'Ritchie, NEW', 'Sane, MCI', 'Drinkwater, CHE', 'Lukaku, MUN', 'Arnautovic, WHU', 'Kane, TOT'] 
Gen: 2, Fitness: 14473.05164425615, Best: ['Ryan, BHA', 'Evans, LEI', 'Schelotto, BHA', 'Long, BUR', 'Aurier, TOT', 'Ritchie, NEW', 'Sane, MCI', 'Drinkwater, CHE', 'Lukaku, MUN', 'Arnautovic, WHU', 'Kane, TOT'] 
Gen: 3, Fitness: 15432.176459895665, Best: ['Ryan, BHA', 'Valencia, MUN', 'Stephens, SOU', 'Baines, EVE', 'Williams, EVE', 'Sane, MCI', 'Mkhitaryan, ARS', 'Winks, TOT', 'Lukaku, MUN', 'Arnautovic, WHU', 'Kane, TOT'] 
Gen: 4, Fitness: 16643.18833089172, Best: ['De Gea, MUN', 'Bailly, MUN', 'Stephens, SOU', 'Long, BUR', 'Baines, EVE', 'Winks, TOT', 'Sane, MCI', 'M

## 3-6-1

In [33]:
formation = (3,6,1)
max_gens = 100
pop_size = 30
p_crossover = .99
p_mutation = 1.0/64
week = 0

foo = search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week)

print('\n ***************************************************************************** \n' 
'                        Best Lineup for week {0}                                    \n' 
'                         Formation: {1}-{2}-{3}                                     \n' 
'***************************************************************************** \n'                    
'GKP: {4} \n'
'DEF: {5} \n'
'MID: {6} \n'
'FWD: {7}'.format(week, formation[0], formation[1], formation[2], foo['lineup'][0], 
                                                                  foo['lineup'][1:(formation[0]+1)],
                                                                  foo['lineup'][(formation[0]+1):(formation[0]+formation[1]+1)],
                                                                  foo['lineup'][(formation[0]+formation[1]+1):]))

Gen: 0, Fitness: 9681.538945478194, Best: ['Lossl, HUD', 'Robertson, LIV', 'Kabasele, WAT', 'Souare, CRY', 'Lookman, EVE', 'Ince, HUD', 'Lallana, LIV', 'Sissoko, TOT', 'Son, TOT', 'Salah, LIV', 'King, BOU'] 
Gen: 1, Fitness: 9681.538945478194, Best: ['Lossl, HUD', 'Robertson, LIV', 'Kabasele, WAT', 'Souare, CRY', 'Lookman, EVE', 'Ince, HUD', 'Lallana, LIV', 'Sissoko, TOT', 'Son, TOT', 'Salah, LIV', 'King, BOU'] 
Gen: 2, Fitness: 10065.262866683534, Best: ['Lossl, HUD', 'Schlupp, CRY', 'Moses, CHE', 'Souare, CRY', 'Lookman, EVE', 'Ince, HUD', 'Lallana, LIV', 'Sissoko, TOT', 'Son, TOT', 'Salah, LIV', 'King, BOU'] 
Gen: 3, Fitness: 10923.705683076925, Best: ['Pope, BUR', 'Lowton, BUR', 'Oxford, WHU', 'Baines, EVE', 'Lookman, EVE', 'Ince, HUD', 'Lallana, LIV', 'Sissoko, TOT', 'Son, TOT', 'Salah, LIV', 'Calvert-Lewin, EVE'] 
Gen: 4, Fitness: 10923.705683076925, Best: ['Pope, BUR', 'Lowton, BUR', 'Oxford, WHU', 'Baines, EVE', 'Lookman, EVE', 'Ince, HUD', 'Lallana, LIV', 'Sissoko, TOT', 'Son,

## 4-5-1

In [34]:
formation = (4,5,1)
max_gens = 100
pop_size = 30
p_crossover = .99
p_mutation = 1.0/64
week = 0

foo = search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week)

print('\n ***************************************************************************** \n' 
'                        Best Lineup for week {0}                                    \n' 
'                         Formation: {1}-{2}-{3}                                     \n' 
'***************************************************************************** \n'                    
'GKP: {4} \n'
'DEF: {5} \n'
'MID: {6} \n'
'FWD: {7}'.format(week, formation[0], formation[1], formation[2], foo['lineup'][0], 
                                                                  foo['lineup'][1:(formation[0]+1)],
                                                                  foo['lineup'][(formation[0]+1):(formation[0]+formation[1]+1)],
                                                                  foo['lineup'][(formation[0]+formation[1]+1):]))

Gen: 0, Fitness: 8214.484735249109, Best: ['Ryan, BHA', 'Mings, BOU', 'Chilwell, LEI', 'Bednarek, SOU', 'David Luiz, CHE', 'Drinkwater, CHE', 'Ward-Prowse, SOU', 'Fraser, BOU', 'Vlasic, EVE', 'Sterling, MCI', 'Giroud, CHE'] 
Gen: 1, Fitness: 8791.665281119005, Best: ['Schmeichel, LEI', 'Malone, HUD', 'Moses, CHE', 'Mings, BOU', 'Evans, LEI', 'Lamela, TOT', 'Arter, BOU', 'Diaz, MCI', 'De Bruyne, MCI', 'Amartey, LEI', 'Firmino, LIV'] 
Gen: 2, Fitness: 10456.754875835506, Best: ['Ryan, BHA', 'Malone, HUD', 'Smalling, MUN', 'Mings, BOU', 'Moses, CHE', 'Lamela, TOT', 'Arter, BOU', 'Ki Sung-yueng, NEW', 'Diaz, MCI', 'De Bruyne, MCI', 'Firmino, LIV'] 
Gen: 3, Fitness: 10456.754875835506, Best: ['Ryan, BHA', 'Malone, HUD', 'Smalling, MUN', 'Mings, BOU', 'Moses, CHE', 'Lamela, TOT', 'Arter, BOU', 'Ki Sung-yueng, NEW', 'Diaz, MCI', 'De Bruyne, MCI', 'Firmino, LIV'] 
Gen: 4, Fitness: 10827.832408252856, Best: ['Pope, BUR', 'Moses, CHE', 'Otamendi, MCI', 'Benalouane, LEI', 'Hadergjonaj, HUD', 'Fel

## 4-4-2

In [35]:
formation = (4,4,2)
max_gens = 100
pop_size = 30
p_crossover = .99
p_mutation = 1.0/64
week = 0

foo = search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week)

print('\n ***************************************************************************** \n' 
'                        Best Lineup for week {0}                                    \n' 
'                         Formation: {1}-{2}-{3}                                     \n' 
'***************************************************************************** \n'                    
'GKP: {4} \n'
'DEF: {5} \n'
'MID: {6} \n'
'FWD: {7}'.format(week, formation[0], formation[1], formation[2], foo['lineup'][0], 
                                                                  foo['lineup'][1:(formation[0]+1)],
                                                                  foo['lineup'][(formation[0]+1):(formation[0]+formation[1]+1)],
                                                                  foo['lineup'][(formation[0]+formation[1]+1):]))

Gen: 0, Fitness: 10735.860645584602, Best: ['Hennessey, CRY', 'Jones, MUN', 'Janmaat, WAT', 'Smalling, MUN', 'Young, MUN', 'Alli, TOT', 'James, LEI', 'Elneny, ARS', 'Surman, BOU', 'Perez, NEW', 'Kane, TOT'] 
Gen: 1, Fitness: 10735.860645584602, Best: ['Hennessey, CRY', 'Jones, MUN', 'Janmaat, WAT', 'Smalling, MUN', 'Young, MUN', 'Alli, TOT', 'James, LEI', 'Elneny, ARS', 'Surman, BOU', 'Perez, NEW', 'Kane, TOT'] 
Gen: 2, Fitness: 10735.860645584602, Best: ['Hennessey, CRY', 'Jones, MUN', 'Janmaat, WAT', 'Smalling, MUN', 'Young, MUN', 'Alli, TOT', 'James, LEI', 'Elneny, ARS', 'Surman, BOU', 'Perez, NEW', 'Kane, TOT'] 
Gen: 3, Fitness: 11539.242874299704, Best: ['Hennessey, CRY', 'Young, MUN', 'Simpson, BOU', 'Walker, MCI', 'Smalling, MUN', 'Mkhitaryan, ARS', 'Ramsey, ARS', 'Fernandinho, MCI', 'Willian, CHE', 'Mousset, BOU', 'Firmino, LIV'] 
Gen: 4, Fitness: 11539.242874299704, Best: ['Hennessey, CRY', 'Young, MUN', 'Simpson, BOU', 'Walker, MCI', 'Smalling, MUN', 'Mkhitaryan, ARS', 'Ramse

# Best Team for Lowest Player Cost